In [2]:
import sys
import os
sys.path.insert(0, 'D:\\PROGRAMMING\\mlflow')

In [3]:
import mlflow
import mlflow.pytorch
from dataclasses import dataclass, asdict

import torch
import torch.nn as nn
import numpy as np

from time import sleep

print(os.path.dirname(mlflow.__file__))

D:\PROGRAMMING\mlflow\mlflow


In [5]:
@dataclass
class Config:
    EPOCHS: int = 100
    LEARNING_RATE: int = 0.1

CONFIG = Config()

In [6]:
mlflow.set_experiment('Test Experiment')
def mlflow_run_decorator(run_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            mlflow.start_run(run_name=run_name)
            try:
                result = func(*args, **kwargs)
                mlflow.set_tag("Status", "SUCCEESS")
            except Exception as e:
                mlflow.log_param("Exception", e)
                mlflow.set_tag("Status", "FAIL")
                raise e
            finally:
                mlflow.end_run()
            return result
        return wrapper
    return decorator

2024/08/04 01:05:44 INFO mlflow.tracking.fluent: Experiment with name 'Test Experiment' does not exist. Creating a new experiment.


In [7]:
X = np.random.rand(100, 1)
y = 1 + 2 * X + .1 * np.random.randn(100, 1)

In [8]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [10]:
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG.LEARNING_RATE)

In [12]:
@mlflow_run_decorator(run_name='Check Sync')
def train_model():
    mlflow.log_params(asdict(CONFIG))
    
    for epoch in range(CONFIG.EPOCHS):
        inputs = torch.from_numpy(X.astype(np.float32))
        labels = torch.from_numpy(y.astype(np.float32))

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        mlflow.log_metric('loss', loss.item(), step=epoch)
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}/{CONFIG.EPOCHS}, Loss: {loss.item()}')
    
        sleep(0)

In [13]:
train_model()

Epoch 0/1000, Loss: 3.3055307865142822
Epoch 100/1000, Loss: 0.03637167066335678
Epoch 200/1000, Loss: 0.011822047643363476
Epoch 300/1000, Loss: 0.009042587131261826
Epoch 400/1000, Loss: 0.008727899752557278
Epoch 500/1000, Loss: 0.008692270144820213
Epoch 600/1000, Loss: 0.008688240312039852
Epoch 700/1000, Loss: 0.008687784895300865
Epoch 800/1000, Loss: 0.008687730878591537
Epoch 900/1000, Loss: 0.00868772342801094


In [ ]:
mlflow.